In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
import pandas as pd
import re
from datetime import datetime

from sklearn import compose
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import *

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
%matplotlib inline

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from rfpimp import *

import warnings
warnings.filterwarnings("ignore")

## Imports and EDA

In [2]:
df = pd.read_csv("./wine-reviews/winemag-data-130k-v2.csv", index_col=0)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
def make_pipelines(n_estimators):
    "Create a single pipeline that processing the data and then fits the regressor." 
        
    numeric_features = ['price']
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('scl', StandardScaler())])
    
    numeric_transformer_tree = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

    preprocessor = compose.ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features)])
    preprocessor_tree = compose.ColumnTransformer(transformers=[('num', numeric_transformer_tree, numeric_features)])


    
    pipe_lr = Pipeline([('preprocessor', preprocessor), 
                        ('regr', LinearRegression(n_jobs=-1))])
    
    pipe_lasso = Pipeline([('preprocessor', preprocessor),
                           ('regr', Lasso(alpha=0.2))])
    
    pipe_rf = Pipeline([('preprocessor', preprocessor_tree),
                        ('regr', RandomForestRegressor(n_estimators=n_estimators, 
                                                      min_samples_leaf=.1, 
                                                      random_state=42, 
                                                      n_jobs=-1))])

    pipe_xtr = Pipeline([('preprocessor', preprocessor_tree),
                           ('regr', ExtraTreesRegressor(n_estimators=n_estimators*5, 
                                                        random_state=42,
                                                        n_jobs=-1))])
    
    pipe_ada = Pipeline([('preprocessor', preprocessor_tree),
                           ('regr', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                                                           min_samples_leaf=.1),
                                                      n_estimators=n_estimators,
                                                      random_state=42))])
    
    pipeline = [pipe_lr, pipe_lasso, pipe_rf, pipe_xtr]
    return pipeline

In [4]:
def evaluate(y_test, y_pred):

    results = {'Median Absolute Error' : round(median_absolute_error(y_test, y_pred), 3),
               'Mean Absolute Error' : round(mean_absolute_error(y_test, y_pred), 3),
               'R-Sq' : round(r2_score(y_test, y_pred), 2),
               'MSE' : round(mean_squared_error(y_test, y_pred),2),
               'Max Error': round(max_error(y_test, y_pred),2)}
                             
    return results

# Baselines before Feature Engineering

In [5]:
X = df[['price']]
y = df['points']

In [6]:
pipelines = make_pipelines(n_estimators=30)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [8]:
def pipeline_results(pipelines, X_train, X_test, y_train, y_test):
    models = []
    results_df = pd.DataFrame(columns=['MedAE','MeanAE','R-Squared','MSE','Max Error'])
    
    for p in pipelines:
        print('.',end='')
        p.fit(X_train, y_train)
        models.append(p)

        y_pred = p.predict(X_test)

        metrics = list(evaluate(y_test, y_pred).values())    
        results_df.loc[len(results_df)] = metrics

    results_df = results_df.T
    results_df.columns = ['Linear','Lasso','RF','XT']
    return models, results_df.T

In [9]:
%%time
models, results = pipeline_results(pipelines, X_train, X_test, y_train, y_test)
results

....CPU times: user 6.26 s, sys: 113 ms, total: 6.37 s
Wall time: 3.15 s


,MedAE,MeanAE,R-Squared,MSE,Max Error
Linear,1.900,2.229,0.15,7.94,58.63
Lasso,1.938,2.256,0.15,7.96,48.58
RF,1.624,1.994,0.33,6.27,10.50
XT,1.616,1.955,0.36,6.04,10.84


## Adding One Hot Encoding

In [18]:
def make_pipelines_2(n_estimators):
    "Create a single pipeline that processing the data and then fits the regressor." 
        
    numeric_features = ['price']
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('scl', StandardScaler())])

    numeric_transformer_tree = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

    categorical_features = ['taster_name','country','region_2']
    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                              ('hot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = compose.ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                           ('cat', categorical_transformer, categorical_features)])

    preprocessor_tree = compose.ColumnTransformer(transformers=[('num', numeric_transformer_tree, numeric_features),
                                                           ('cat', categorical_transformer, categorical_features)])
    pipe_lr = Pipeline([('preprocessor', preprocessor), 
                        ('regr', LinearRegression(n_jobs=-1))])
    
    pipe_lasso = Pipeline([('preprocessor', preprocessor),
                           ('regr', Lasso(alpha=0.2))])
    
    pipe_rf = Pipeline([('preprocessor', preprocessor_tree),
                        ('regr', RandomForestRegressor(n_estimators=n_estimators, 
                                                      min_samples_leaf=.1, 
                                                      random_state=42, 
                                                      n_jobs=-1))])

    pipe_xtr = Pipeline([('preprocessor', preprocessor_tree),
                           ('regr', ExtraTreesRegressor(n_estimators=n_estimators*5, 
                                                        min_samples_leaf=.1,
                                                        random_state=42,
                                                        n_jobs=-1))])
    
    pipe_ada = Pipeline([('preprocessor', preprocessor_tree),
                           ('regr', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                                                           min_samples_leaf=.1),
                                                      n_estimators=n_estimators,
                                                      random_state=42))])
                                                  
    pipeline = [pipe_lr, pipe_lasso, pipe_rf, pipe_xtr]
    return pipeline

In [12]:
pipelines_2 = make_pipelines_2(30)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
%%time
models, results = pipeline_results(pipelines_2, X_train, X_test, y_train, y_test)
results

....CPU times: user 6min 17s, sys: 15.6 s, total: 6min 33s
Wall time: 2min 57s


,MedAE,MeanAE,R-Squared,MSE,Max Error
Linear,1.792,2.088,0.25,7.00,54.81
Lasso,1.938,2.256,0.15,7.96,48.58
RF,1.624,1.994,0.33,6.27,10.50
XT,1.500,1.827,0.42,5.42,11.00


In [19]:
pipelines_2 = make_pipelines_2(30)

In [20]:
%%time
models, results = pipeline_results(pipelines_2, X_train, X_test, y_train, y_test)
results

....CPU times: user 1min 7s, sys: 15.2 s, total: 1min 22s
Wall time: 1min 20s


,MedAE,MeanAE,R-Squared,MSE,Max Error
Linear,1.792,2.088,0.25,7.00,54.81
Lasso,1.938,2.256,0.15,7.96,48.58
RF,1.624,1.994,0.33,6.27,10.50
XT,2.061,2.391,0.07,8.69,11.70


## Feature Engineering

### Varieties

In [21]:
# too many columns to add
hot_varieties = pd.get_dummies(df['variety'])
hot_varieties 

,Abouriou,Agiorgitiko,Aglianico,Aidani,Airen,Albana,Albanello,Albariño,Albarossa,Aleatico,...,Yapincak,Zelen,Zibibbo,Zierfandler,Zierfandler-Rotgipfler,Zinfandel,Zlahtina,Zweigelt,Çalkarası,Žilavka
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
hot_varieties.corrwith(df['points']).abs().sort_values(ascending=False)

Pinot Noir                   0.106986
Rosé                         0.088433
Nebbiolo                     0.088123
Riesling                     0.067290
Sauvignon Blanc              0.066725
                               ...   
Merlot-Syrah                 0.000068
Duras                        0.000045
Pinot Bianco                 0.000010
Greco                        0.000009
Malbec-Cabernet Sauvignon    0.000008
Length: 707, dtype: float64

In [23]:
# select only the top 25 by variation
varieties_25 = hot_varieties.corrwith(df['points']).abs().sort_values(ascending=False).head(25).index
df.loc[~df.variety.isin(varieties_25), 'variety'] = np.nan

In [26]:
df.variety

0         White Blend
1                 NaN
2                 NaN
3            Riesling
4          Pinot Noir
             ...     
129966       Riesling
129967     Pinot Noir
129968            NaN
129969            NaN
129970            NaN
Name: variety, Length: 129971, dtype: object

### Parsing Vintage Year

In [27]:
regex = re.compile('[^0-9]')

title_words = [t.split() for t in df.title]  
rem_char_titles = []

for t in title_words:
    no_char_t = []
    for w in t:
        cw = regex.sub('',w)
        if len(cw) == 4:
            no_char_t.append(cw)
            
    rem_char_titles.append(no_char_t)

In [28]:
vintages = []
for i,l in enumerate(rem_char_titles):
    if not l:
        vintages.append(np.nan)
    elif len(l) == 1:
        year = int(l[0])
        if year >= 1900:
            vintages.append(year)
        else:
            vintages.append(1900)
    else:
        l = map(int,l)
        for i in l:
            if i > 1980 and i <= 2017:
                vintages.append(i)
                break
                
vintages = pd.Series(vintages) 

In [29]:
vintages

0         2013.0
1         2011.0
2         2013.0
3         2013.0
4         2012.0
           ...  
129966    2013.0
129967    2004.0
129968    2013.0
129969    2012.0
129970    2012.0
Length: 129971, dtype: float64

In [32]:
vintage_df = pd.DataFrame({'Age' : max(vintages) - vintages}).fillna(vintages.median())
vintage_df['Age Bucket'] = vintage_df['Age'] // 5
vintage_df.loc[vintage_df['Age Bucket'] > 10, 'Age Bucket'] = np.nan
vintage_df

,Age,Age Bucket
0,4.0,0.0
1,6.0,1.0
2,4.0,0.0
3,4.0,0.0
4,5.0,1.0
...,...,...
129966,4.0,0.0
129967,13.0,2.0
129968,4.0,0.0
129969,5.0,1.0


In [33]:
vintage_df['Age Bucket'].value_counts()

1.0     57789
0.0     45211
2.0     18810
3.0      3007
4.0       446
5.0        36
6.0        14
10.0        9
7.0         5
9.0         5
8.0         3
Name: Age Bucket, dtype: int64

### Outlier Removal

In [52]:
scaler = StandardScaler()
z_price = scaler.fit_transform(df.price.fillna(df.price.median()).to_numpy().reshape(-1,1))

In [53]:
z_price = pd.DataFrame({'Z Price' : z_price.flatten()})
z_price.loc[z_price['Z Price'] > 5, 'Z Price'] = 5
z_price.loc[z_price['Z Price'] < -5, 'Z Price'] = -5
z_price

,Z Price
0,-0.243193
1,-0.495310
2,-0.520521
3,-0.545733
4,0.765272
...,...
129966,-0.167559
129967,1.017388
129968,-0.117135
129969,-0.066712


In [47]:
min(z_price['Z Price']), max(z_price['Z Price'])

(-0.7726377396161901, 5.0)

In [58]:
price_inliers = pd.DataFrame({'Price Inliers': scaler.inverse_transform(z_price).flatten()})
price_inliers

,Price Inliers
0,25.0
1,15.0
2,14.0
3,13.0
4,65.0
...,...
129966,28.0
129967,75.0
129968,30.0
129969,32.0


### Vader:  Sentiment Analysis

In [59]:
analyzer = SentimentIntensityAnalyzer()

def analyze_polarity(col):
    polarity = {'Neg Sent':[], 'Neu Sent':[], 'Pos Sent':[], 'Comp Sent':[]}
    for i in col:
        scores = analyzer.polarity_scores(i)
        polarity['Neg Sent'].append(scores['neg'])
        polarity['Neu Sent'].append(scores['neu'])
        polarity['Pos Sent'].append(scores['pos'])
        polarity['Comp Sent'].append(scores['compound'])
    return polarity

In [60]:
%%time
sentiments = analyze_polarity(df.description)

CPU times: user 1min 29s, sys: 648 ms, total: 1min 30s
Wall time: 1min 33s


In [61]:
sentiments = pd.DataFrame(sentiments)
sentiments.tail(3)

,Neg Sent,Neu Sent,Pos Sent,Comp Sent
129968,0.072,0.865,0.063,0.1548
129969,0.000,0.891,0.109,0.5267
129970,0.047,0.723,0.230,0.7003


### Description Adjectives

In [62]:
adjectives =  ['fruit', 'strong', 'tangy', 'bitter', 'alcohol', 
               'floral', 'acidity', 'tobacco','tannin', 'ripe', 
               'spice', 'oak', 'rich', 'dry', 'crisp', 'sweet', 
               'vanilla', 'full', 'tropical', 'strong', 'bitter']

In [63]:
for adj in adjectives:
    df[adj] = 0
    df.loc[df.description.str.contains(adj), adj] = 1

In [64]:
adj_df = df[adjectives]
adj_df

,fruit,strong,tangy,bitter,alcohol,floral,acidity,tobacco,tannin,ripe,...,oak,rich,dry,crisp,sweet,vanilla,full,tropical,strong,bitter
0,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
129967,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
129968,1,0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0
129969,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0


### Added the extra features...
- varieties one hot
- vintage year numeric column
- vintage 5 year bucket categorical column
- outlier removed prices at 5 sigmas
- vader sentiment numeric columns
- adjectives one hot

In [76]:
def make_pipelines_3(n_estimators):
    "Create a single pipeline that processing the data and then fits the regressor." 
        
    numeric_features = ['Age','Neg Sent','Neu Sent','Pos Sent','Comp Sent','Price Inliers']
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('scl', StandardScaler())])
    
    numeric_transformer_tree = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

    categorical_features = ['taster_name','country','region_2','Age Bucket', 'variety']
    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                              ('hot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = compose.ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                           ('cat', categorical_transformer, categorical_features)])
    
    preprocessor_tree = compose.ColumnTransformer(transformers=[('num', numeric_transformer_tree, numeric_features),
                                                                ('cat', categorical_transformer, categorical_features)])
    
    
    pipe_lr = Pipeline([('preprocessor', preprocessor), 
                        ('regr', LinearRegression(n_jobs=-1))])
    
    pipe_lasso = Pipeline([('preprocessor', preprocessor),
                           ('regr', Lasso(alpha=0.2))])
    
    pipe_rf = Pipeline([('preprocessor', preprocessor_tree),
                        ('regr', RandomForestRegressor(n_estimators=n_estimators, 
                                                      min_samples_leaf=.1, 
                                                      random_state=42, 
                                                      n_jobs=-1))])

    pipe_xtr = Pipeline([('preprocessor', preprocessor_tree),
                           ('regr', ExtraTreesRegressor(n_estimators=n_estimators*5, 
                                                        random_state=42,
                                                        n_jobs=-1))])
    
    pipe_ada = Pipeline([('preprocessor', preprocessor_tree),
                           ('regr', AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                                                           min_samples_leaf=.1),
                                                      n_estimators=n_estimators,
                                                      random_state=42))])
    
    pipeline = [pipe_lr, pipe_lasso, pipe_rf, pipe_xtr]
    return pipeline

In [78]:
X = df.drop('price', axis=1)
X_fe = pd.concat([X, adj_df, vintage_df, price_inliers, sentiments], axis=1)

In [79]:
X_train_fe, X_test_fe, y_train, y_test = train_test_split(X_fe, y, test_size=0.20, random_state=42)

In [82]:
pipelines_3 = make_pipelines_3(3)

In [83]:
%%time
models, results = pipeline_results(pipelines_3, X_train_fe, X_test_fe, y_train, y_test)
results

....CPU times: user 9min 47s, sys: 18.6 s, total: 10min 5s
Wall time: 4min 20s


,MedAE,MeanAE,R-Squared,MSE,Max Error
Linear,1.531,1.815,0.44,5.20,11.83
Lasso,1.746,2.030,0.33,6.32,12.57
RF,1.747,1.957,0.35,6.06,10.10
XT,1.133,1.492,0.56,4.13,11.73


In [84]:
pipelines_3 = make_pipelines_3(30)

In [ ]:
%%time
models, results = pipeline_results(pipelines_3, X_train_fe, X_test_fe, y_train, y_test)
results

....

## Model Selection

In [ ]:
m = models[3]

In [ ]:
imp = importances(m, X_test_fe, y_test)
viz = plot_importances(imp)
viz.view()

In [ ]:
imp.head(15)

In [ ]:
imp.tail(15)

## Reduced Model

In [ ]:
features_50 = imp.head(50).index

In [ ]:
X_reduced = X_full_features[features_50]

In [ ]:
X_train_reduced, X_test_reduced, y_train, y_test = train_test_split(X_reduced, y, test_size=0.20, random_state=42)

In [ ]:
%%time
rf.fit(X_train_reduced, y_train)
y_pred_reduced_rf = rf.predict(X_test_reduced)
rf_results_reduced = evaluate(y_test, y_pred_reduced_rf)

In [ ]:
rf_results_reduced

In [ ]:
hyperparameters = {"min_samples_leaf" : np.arange(5,100,5),
                    "n_estimators" : np.arange(10,200,20)}

In [ ]:
random_cv = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1, warm_start=True), 
                              hyperparameters, 
                              cv=3, 
                              n_iter=3, 
                              verbose=2)

In [ ]:
random_cv.fit(X_train_reduced, y_train)

In [ ]:
y_pred_cv = random_cv.predict(X_test_reduced)

In [ ]:
cv_results = evaluate(y_test, y_pred_cv)
cv_results

In [ ]:
y_pred_cv = random_cv.predict(X_test_reduced)